In [6]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.settings import Settings
from llama_index.core import Document
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client.http.models import Distance, VectorParams, Filter, FieldCondition, MatchValue
import fitz  # PyMuPDF
import uuid
import logging
from typing import Optional, Tuple, List, Dict, Any, Union
from datetime import datetime
from playwright.sync_api import sync_playwright
import requests
from bs4 import BeautifulSoup
from readability import Document as ReadabilityDoc

# Configure logging
logger = logging.getLogger(__name__)


In [23]:
def render_with_headless(url: str) -> str:
    """
    Render the given URL via Playwright and return fully rendered HTML.
    """
    with sync_playwright() as pw:
        browser = pw.chromium.launch(
            headless=True,
            args=["--no-sandbox", "--disable-dev-shm-usage"]
        )
        page = browser.new_page()
        page.goto(url, wait_until="networkidle")
        html = page.content()
        browser.close()
        return html

def extract_document_from_url(
    url: str,
    source_id: str,
    tags: List[str],
    uploaded_at: str,
    extras: Optional[Dict[str, Any]] = None
) -> Document:
    logger.info("Starting URL extraction: %s", url)

    # 1) Initial fetch via requests
    try:
        resp = requests.get(
            url,
            headers={"User-Agent": "Mozilla/5.0"},
            timeout=10
        )
        resp.raise_for_status()
        html_source = resp.text
        logger.info("Fetched via requests: %s", url)
    except Exception as e:
        logger.warning("Requests fetch failed for %s: %s", url, e)
        html_source = render_with_headless(url)

    soup = BeautifulSoup(html_source, 'html.parser')

    # Remove boilerplate tags
    for tag_name in [
        "script", "style", "nav", "header", "footer",
        "form", "aside", "iframe", "noscript", "meta", "link"
    ]:
        for tag in soup.find_all(tag_name):
            tag.decompose()

    # 2) Try readability for clean extraction
    try:
        rd = ReadabilityDoc(html_source)
        summary_html = rd.summary()
        rd_soup = BeautifulSoup(summary_html, 'html.parser')
        text_body = rd_soup.get_text(separator='\n', strip=True)
        title = rd.title().strip() if rd.title() else ''
        if text_body:
            full_text = f"{title}\n\n{text_body}" if title else text_body
            logger.info("Readability extraction succeeded for %s", url)
            return Document(
                text=full_text,
                metadata={
                    "source_id": source_id,
                    "url": url,
                    "type": "url",
                    "tags": tags,
                    "extras": extras,
                    "uploaded_at": uploaded_at
                }
            )
    except Exception:
        logger.debug("Readability extraction failed for %s", url)

    # 3) Heuristic: pick the largest text-heavy block
    def find_main_block(soup: BeautifulSoup) -> Optional[BeautifulSoup]:
        candidates = soup.find_all(['main', 'article', 'section', 'div'], recursive=True)
        best = None
        best_len = 0
        for el in candidates:
            text = el.get_text(separator=' ', strip=True)
            if len(text) < 200:
                continue
            # skip link-heavy blocks
            links = el.find_all('a')
            if links and len(''.join(a.get_text() for a in links)) / len(text) > 0.3:
                continue
            if len(text) > best_len:
                best, best_len = el, len(text)
        return best or soup.body or soup

    main_block = find_main_block(soup)
    raw_text = main_block.get_text(separator='\n', strip=True)
    lines = [line for line in raw_text.splitlines() if line.strip()]
    clean_text = '\n\n'.join(lines)

    # 4) Fallback to headless if too little text
    if len(clean_text) < 200:
        logger.info("Heuristic extraction too small, using headless for %s", url)
        rendered = render_with_headless(url)
        soup = BeautifulSoup(rendered, 'html.parser')
        for tag_name in [
            "script", "style", "nav", "header", "footer",
            "form", "aside", "iframe", "noscript", "meta", "link"
        ]:
            for tag in soup.find_all(tag_name):
                tag.decompose()
        main_block = find_main_block(soup)
        raw_text = main_block.get_text(separator='\n', strip=True)
        lines = [ln for ln in raw_text.splitlines() if ln.strip()]
        clean_text = '\n\n'.join(lines)

    if not clean_text:
        raise ValueError(f"No text could be extracted from {url}")

    title_tag = soup.title.string.strip() if soup.title else ''
    if title_tag:
        clean_text = f"{title_tag}\n\n{clean_text}"

    logger.info("Extraction succeeded for %s (length=%d)", url, len(clean_text))
    return Document(
        text=clean_text,
        metadata={
            "source_id": source_id,
            "url": url,
            "type": "url",
            "tags": tags,
            "extras": extras,
            "uploaded_at": uploaded_at
        }
    )


In [24]:
url = "https://www.rrq.gouv.qc.ca/fr/programmes/soutien_enfants/supplement/Pages/supplement.aspx"
source_id = "id1"
tags = None
extras = None
uploaded_at = datetime.utcnow().isoformat() + "Z"

document = extract_document_from_url(url, source_id, tags, uploaded_at, extras)



In [25]:
document.__dict__

{'id_': 'fa2bf605-27f9-4eb5-9fe7-6a5485ed6aa0',
 'embedding': None,
 'metadata': {'source_id': 'id1',
  'url': 'https://www.rrq.gouv.qc.ca/fr/programmes/soutien_enfants/supplement/Pages/supplement.aspx',
  'type': 'url',
  'tags': None,
  'extras': None,
  'uploaded_at': '2025-05-27T15:39:00.606404Z'},
 'excluded_embed_metadata_keys': [],
 'excluded_llm_metadata_keys': [],
 'relationships': {},
 'metadata_template': '{key}: {value}',
 'metadata_separator': '\n',
 'text_resource': MediaResource(embeddings=None, data=None, text="Retraite Québec - Supplément pour enfant handicapé\n\nLe supplément pour enfant\xa0handicapé\nLe supplément pour enfant handicapé\xa0(SEH) est  une aide financière pour les parents qui ont à leur charge un enfant de\nmoins  de 18\xa0ans\nqui présente une\ndéficience physique ou un trouble  des fonctions mentales qui le limite de façon importante dans la réalisation de  ses habitudes de vie\npar rapport à ce qui est attendu d'un enfant de son  âge. La durée prévis

In [26]:
print(document.text_resource.text)

Retraite Québec - Supplément pour enfant handicapé

Le supplément pour enfant handicapé
Le supplément pour enfant handicapé (SEH) est  une aide financière pour les parents qui ont à leur charge un enfant de
moins  de 18 ans
qui présente une
déficience physique ou un trouble  des fonctions mentales qui le limite de façon importante dans la réalisation de  ses habitudes de vie
par rapport à ce qui est attendu d'un enfant de son  âge. La durée prévisible des incapacités doit être
d'au  moins un an
. Nous déterminons l'admissibilité de votre enfant au  SEH selon certains
critères
.
Une équipe de professionnelles et professionnels  de la santé de Retraite Québec évalue les demandes de SEH. Elle est composée de  médecins, d'orthophonistes, de psychologues et de membres du personnel infirmier.
Cette équipe détermine si l'enfant est admissible  au SEH à partir de l'ensemble des renseignements contenus dans son dossier. En  plus des renseignements fournis par la famille qui dépose une demande, 

In [27]:
resp = requests.get(
    url,
    headers={"User-Agent": "Mozilla/5.0"},
    timeout=10
)
resp.raise_for_status()
html_source = resp.text
logger.info("Fetched via requests: %s", url)


In [30]:
print(html_source)



<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

    

    <html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#" dir="ltr" lang="fr" xml:lang="fr"><head><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1" /><meta name="format-detection" content="telephone=no" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="GENERATOR" content="Microsoft SharePoint" /><meta http-equiv="Content-type" content="text/html; charset=utf-8" /><meta http-equiv="Expires" content="0" /><meta name="google-site-verification" content="ffUnqBSAk1q4b0HWrxk3Y1RDxPsKFQoPEO6gE_xuTc8" /><title>
	Retraite Québec - Supplément pour enfant handicapé
</title><!-- Ajout RRQ : Doit-être placé avant le PlaceHolderAdditionalPageHead -->
<meta http-equiv="Content-Type" content="text/html; charset=utf-8" />
<meta http-equiv="Content-Language" content="fr"

In [31]:
soup = BeautifulSoup(html_source, 'html.parser')
print(soup)


<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#"><head><meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="telephone=no" name="format-detection"/><meta content="width=device-width, initial-scale=1" name="viewport"/><meta content="Microsoft SharePoint" name="GENERATOR"/><meta content="text/html; charset=utf-8" http-equiv="Content-type"/><meta content="0" http-equiv="Expires"/><meta content="ffUnqBSAk1q4b0HWrxk3Y1RDxPsKFQoPEO6gE_xuTc8" name="google-site-verification"/><title>
	Retraite Québec - Supplément pour enfant handicapé
</title><!-- Ajout RRQ : Doit-être placé avant le PlaceHolderAdditionalPageHead -->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="fr" http-equiv="Content-Language"/>
<meta content="L

In [32]:
# Remove boilerplate tags
for tag_name in [
    "script", "style", "nav", "header", "footer",
    "form", "aside", "iframe", "noscript", "meta", "link"
]:
    for tag in soup.find_all(tag_name):
        tag.decompose()


In [34]:
print(soup)


<?xml version="1.0" encoding="utf-8"?>
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html dir="ltr" lang="fr" xml:lang="fr" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://ogp.me/ns/fb#"><head><title>
	Retraite Québec - Supplément pour enfant handicapé
</title><!-- Ajout RRQ : Doit-être placé avant le PlaceHolderAdditionalPageHead -->













<!-- Affiche l'icone dans la barre d'adresse -->






<!-- PATCH pour corriger une erreur Javascript en mode internaute -->

<!-- FIN PATCH -->
</head><body>
<div id="fb-root"></div>


<!-- End Meta Pixel Code -->
<!-- =========== RRQ: DEBUT ModeAuthentifie seulement --><!-- ===== STARTER: Adds extra stuff like another form for Survey management ====================================================== -->
</body></html>
<!-- MasterPage 00.0438 -->
<!-- Source 23 -->
<!-- Rendu à l’aide du profil de cache :Internet public (purement anonyme) - 1 heure à : 2025-0

In [48]:
rd = ReadabilityDoc(html_source)
summary_html = rd.summary()
rd_soup = BeautifulSoup(summary_html, 'html.parser')
text_body = rd_soup.get_text(separator='\n', strip=True)
title = rd.title().strip() if rd.title() else ''
if text_body:
    full_text = f"{title}\n\n{text_body}" if title else text_body
    logger.info("Readability extraction succeeded for %s", url)
    d = Document(
        text=full_text,
        metadata={
            "source_id": source_id,
            "url": url,
            "type": "url",
            "tags": tags,
            "extras": extras,
            "uploaded_at": uploaded_at
        }
    )
print(d.text_resource.text)


Retraite Québec - Supplément pour enfant handicapé

Le supplément pour enfant handicapé
Le supplément pour enfant handicapé (SEH) est  une aide financière pour les parents qui ont à leur charge un enfant de
moins  de 18 ans
qui présente une
déficience physique ou un trouble  des fonctions mentales qui le limite de façon importante dans la réalisation de  ses habitudes de vie
par rapport à ce qui est attendu d'un enfant de son  âge. La durée prévisible des incapacités doit être
d'au  moins un an
. Nous déterminons l'admissibilité de votre enfant au  SEH selon certains
critères
.
Une équipe de professionnelles et professionnels  de la santé de Retraite Québec évalue les demandes de SEH. Elle est composée de  médecins, d'orthophonistes, de psychologues et de membres du personnel infirmier.
Cette équipe détermine si l'enfant est admissible  au SEH à partir de l'ensemble des renseignements contenus dans son dossier. En  plus des renseignements fournis par la famille qui dépose une demande, 

In [51]:
rd.summary()

'<html><body><div><div id="ctl00_PlaceHolderMain_ctl05__ControlWrapper_PrCrPlhRichHtmlField" class="ms-rtestate-field" aria-labelledby="ctl00_PlaceHolderMain_ctl05_label"><h1>Le supplément pour enfant\xa0handicapé </h1><p>Le supplément pour enfant handicapé\xa0(SEH) est  une aide financière pour les parents qui ont à leur charge un enfant de\xa0<strong>moins  de 18\xa0ans</strong>\xa0qui présente une \r\n   <strong>déficience physique ou un trouble  des fonctions mentales qui le limite de façon importante dans la réalisation de  ses habitudes de vie</strong> par rapport à ce qui est attendu d\'un enfant de son  âge. La durée prévisible des incapacités doit être \r\n   <strong>d\'au  moins\xa0un\xa0an</strong>. Nous déterminons l\'admissibilité de votre enfant au  SEH selon certains\xa0<a href="https://www.retraitequebec.gouv.qc.ca/fr/enfants/enfant-handicape/supplement-enfant-handicape/Pages/criteres-admissibilite.aspx">critères</a>. </p><p> Une équipe de professionnelles et profession

In [54]:
import trafilatura

# url = "https://example.com/some-article"
downloaded = trafilatura.fetch_url(url)
result = trafilatura.extract(downloaded, include_comments=False, favor_precision=True)
print(result)  # string containing cleaned article text


Le supplément pour enfant handicapé
Le supplément pour enfant handicapé (SEH) est une aide financière pour les parents qui ont à leur charge un enfant de moins de 18 ans qui présente une
déficience physique ou un trouble des fonctions mentales qui le limite de façon importante dans la réalisation de ses habitudes de vie par rapport à ce qui est attendu d'un enfant de son âge. La durée prévisible des incapacités doit être
d'au moins un an. Nous déterminons l'admissibilité de votre enfant au SEH selon certains critères.
Une équipe de professionnelles et professionnels de la santé de Retraite Québec évalue les demandes de SEH. Elle est composée de médecins, d'orthophonistes, de psychologues et de membres du personnel infirmier.
Cette équipe détermine si l'enfant est admissible au SEH à partir de l'ensemble des renseignements contenus dans son dossier. En plus des renseignements fournis par la famille qui dépose une demande, l'équipe de professionnelles et professionnels de la santé de Ret

In [53]:
!pip install trafilatura

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 9.1 MB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 837.9/837.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [trafilatura] [dateparser]
